In [2]:
import pandas as pd
import mariadb
import sys
import numpy as np
import os

In [3]:
allocs = pd.read_csv('/Users/gimli/Desktop/mergeddatabase/allocs/folderdatabase.csv')
print('Number of Rows:', allocs.shape[0])
display(allocs)
accesses = pd.read_csv('/Users/gimli/Desktop/mergeddatabase/Qgroundcontrol Allocs/onlinefilesdatabase.csv')
print('Number of Rows:', accesses.shape[0])
display(accesses)

Number of Rows: 8166440


,Unnamed: 0,alloc_ip,pid,ptr,size,alloc_jiffies,free_jiffies,free_ip,type,command,slab,new_type
0,25,ffffffff81a5ba8f,226,ffff88818dc853f0,176,25383,211939,00000000deadbeef,KMEM_CACHE_BULK,systemd-journal,jbd2_journal_head,KMEM_CACHE_BULK
1,57,ffffffff817204a4,2203,ffff8881b1640000,4096,48840,197120,ffffffff816d38f4,ALLOC_PAGES_FOLIO,bash,no-slab,NaN
2,64,ffffffff817cf984,2348,ffff888185d79b80,320,61904,211790,ffffffff817cf0b7,KMEM_CACHE_BULK,less,filp,NaN
3,65,ffffffff817cf984,2348,ffff888185d78280,320,61906,211768,ffffffff817cf0b7,KMEM_CACHE_BULK,less,filp,NaN
4,66,ffffffff811733c8,2348,ffff8881921a0000,6912,61911,196022,00000000deadbeef,KMEM_CACHE_BULK,less,task_struct,KMEM_CACHE_BULK
...,...,...,...,...,...,...,...,...,...,...,...,...
8166435,8572043,ffffffff8159f82f,2427,ffff88813673db20,32,8657869,8657870,000000000fedbeef,KMEM_CACHE_BULK,QGroundControl,kmalloc-cg-16,KMALLOC_ND
8166436,8572044,ffffffff81a2f5e0,2427,ffff88813673db20,32,8657870,8657874,ffffffff81a2d8ff,KMALLOC_ND,QGroundControl,kmalloc-cg-16,NaN
8166437,8572045,ffffffff81713013,2427,ffff88810a13c400,4352,8657871,8657873,ffffffff81712f82,KMEM_CACHE_BULK,QGroundControl,names_cache,NaN
8166438,8572046,ffffffff81713013,2427,ffff88810a13c400,4352,8657879,8657883,ffffffff81712f82,KMEM_CACHE_BULK,QGroundControl,names_cache,NaN


Number of Rows: 14867950


,Unnamed: 0,alloc_ip,pid,ptr,size,free_jiffies,free_ip,type,command,slab,new_type,alloc_jiffies
0,41,ffffffff8164f7c2,2370,ffff8881a8eb6000,2048,1163.0,000000000fedbeef,KMEM_CACHE_BULK,bash,kmalloc-1k,KMALLOC_ND,1162.0
1,198,18,0,ffffffff8966b468,1,5546.0,00000000deadbeef,UFO_GLOBAL,softirq,no-slab,UFO_GLOBAL,5542.0
2,199,ffffffff8164f7c2,0,ffff8881a8e28c00,256,5714.0,000000000fedbeef,KMEM_CACHE_BULK,softirq,kmalloc-128,KMALLOC_ND,5713.0
3,200,ffffffff825866e4,0,ffff8881a8e28c00,256,9796.0,ffffffff8258ca49,KMALLOC_ND,softirq,kmalloc-128,NaN,5714.0
4,263,ffffffff82799aca,0,ffff88816f098b40,320,428052.0,ffffffff827905f1,KMEM_CACHE_BULK,softirq,skbuff_head_cache,NaN,6351.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14867945,15494152,ffffffff817ea563,2873,ffff888101184400,4352,2240929.0,ffffffff817e889d,KMEM_CACHE_BULK,QGroundControl,names_cache,NaN,2240925.0
14867946,15494153,ffffffff81b3dcef,2873,ffff8881c691b600,48,2240970.0,ffffffff81b3c766,KMEM_CACHE_BULK,QGroundControl,lsm_file_cache,NaN,2240927.0
14867947,15494154,ffffffff817ea563,2873,ffff888101184400,4352,2240968.0,ffffffff817e889d,KMEM_CACHE_BULK,QGroundControl,names_cache,NaN,2240932.0
14867948,15494155,ffffffff8278d9e4,0,ffff8881c6949e00,320,2240958.0,ffffffff82793536,KMEM_CACHE_BULK,softirq,skbuff_head_cache,NaN,2240950.0


In [8]:
common_columns = allocs.columns.intersection(accesses.columns).tolist()

#df = pd.merge(allocs, accesses, how = 'inner', on = common_columns)
df = pd.concat([allocs[common_columns], accesses[common_columns]], axis = 0)
#df = pd.merge(allocs, accesses, on= common_columns)
df = df.drop(columns = 'Unnamed: 0')
print('Number of Rows:', df.shape[0])
df.head()

Number of Rows: 23034390


,alloc_ip,pid,ptr,size,alloc_jiffies,free_jiffies,free_ip,type,command,slab,new_type
0,ffffffff81a5ba8f,226,ffff88818dc853f0,176,25383.0,211939.0,00000000deadbeef,KMEM_CACHE_BULK,systemd-journal,jbd2_journal_head,KMEM_CACHE_BULK
1,ffffffff817204a4,2203,ffff8881b1640000,4096,48840.0,197120.0,ffffffff816d38f4,ALLOC_PAGES_FOLIO,bash,no-slab,NaN
2,ffffffff817cf984,2348,ffff888185d79b80,320,61904.0,211790.0,ffffffff817cf0b7,KMEM_CACHE_BULK,less,filp,NaN
3,ffffffff817cf984,2348,ffff888185d78280,320,61906.0,211768.0,ffffffff817cf0b7,KMEM_CACHE_BULK,less,filp,NaN
4,ffffffff811733c8,2348,ffff8881921a0000,6912,61911.0,196022.0,00000000deadbeef,KMEM_CACHE_BULK,less,task_struct,KMEM_CACHE_BULK


In [10]:
def process_dataframe(df):
    try:
        conn = mariadb.connect(
            host=os.getenv("MYSQL_HOST"),
            port=3306,
            user= os.getenv("MYSQL_USER"),
            database=os.getenv('MYSQL_DB'),
            password=os.getenv("MYSQL_PASSWORD")
        )
    except mariadb.Error as e:
        print(f"Error connecting to the database: {e}")
        sys.exit(1)

    cursor = conn.cursor()

    if 'new_type' in df.columns:
        df = df.drop(columns=["new_type"])

    insert_query = """
        INSERT IGNORE INTO memorizer (alloc_ip, pid, ptr, size, alloc_jiffies, free_jiffies, free_ip, type, command, slab)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """

    batch_size = 1000
    for start in range(0, len(df), batch_size):
        batch = df.iloc[start:start+batch_size].values.tolist()
        cursor.executemany(insert_query, batch)
        conn.commit()
        print(f"Inserted rows {start} to {start + len(batch) - 1}")

    cursor.close()
    conn.close()

if __name__ == "__main__":
    process_dataframe(df)

Inserted rows 0 to 999
Inserted rows 1000 to 1999
Inserted rows 2000 to 2999
Inserted rows 3000 to 3999
Inserted rows 4000 to 4999
Inserted rows 5000 to 5999
Inserted rows 6000 to 6999
Inserted rows 7000 to 7999
Inserted rows 8000 to 8999
Inserted rows 9000 to 9999
Inserted rows 10000 to 10999
Inserted rows 11000 to 11999
Inserted rows 12000 to 12999
Inserted rows 13000 to 13999
Inserted rows 14000 to 14999
Inserted rows 15000 to 15999
Inserted rows 16000 to 16999
Inserted rows 17000 to 17999
Inserted rows 18000 to 18999
Inserted rows 19000 to 19999
Inserted rows 20000 to 20999
Inserted rows 21000 to 21999
Inserted rows 22000 to 22999
Inserted rows 23000 to 23999
Inserted rows 24000 to 24999
Inserted rows 25000 to 25999
Inserted rows 26000 to 26999
Inserted rows 27000 to 27999
Inserted rows 28000 to 28999
Inserted rows 29000 to 29999
Inserted rows 30000 to 30999
Inserted rows 31000 to 31999
Inserted rows 32000 to 32999
Inserted rows 33000 to 33999
Inserted rows 34000 to 34999
Inserted 

In [11]:
print("Number of Missing Values in for each Column: \n" + str(df.isna().sum()))

Number of Missing Values in for each Column: 
alloc_ip                0
pid                     0
ptr                     0
size                    0
alloc_jiffies           0
free_jiffies      7063153
free_ip                 0
type                    0
command                 0
slab                    0
new_type         16991527
dtype: int64
